#### Run notebook in sequential order , dont press run all , instructions will be provided below 

In [1]:
import gc
import os
import sys
import warnings
from pathlib import Path



import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import TimeSeriesSplit

import pickle

warnings.simplefilter("ignore")


# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)


### Methodolgy 

- We first take a look take a look our data and load the relevent files 

- We then develop a neural network where the features are lagged target values (up to 5 day) 
    - to develop this we first build the lag data base 
    
    - we then train and test a base model using this lagged data , using K - fold cross validation, to obtain some preliminary results 
    
    - Hyper parameter tuning is then conducted for drop out rate, hidden layers and learning rate for this model
    
    - then we retrain model using those hyper parameters 
    
    - then obtain a loss score on a held out test set 

- Next we move on to building a nerual network where the features are lagged target values up to 20 days 
    - we build this model using the optmial hyper paramters found earlier
    
    - train the model using K- fold cross validation and obtain a loss score on a held out test set 

- Finally a neural network is constructed where the features are lagged target values of upto 5 days and key Player box scores
    - we build this model using the optmial hyper paramters found earlier
    
    - train the model using K- fold cross validation and obtain a loss score on a held out test set 

### Note on Data 
 - intially data was provided in the form of various csv files , train csv contained various nesting of data as json files. 
 - this were unested and converted to pickle files , which have been used through out the notebook 
 
 - please set path to location of a file before use

 - data files used are : 
     - nextDayPlayerEngagement_train.pkl 
     - players.csv
     - rosters_train.pkl
     - playerBoxScores_train.pkl

In [5]:
# %%capture

# !pip install pandarallel 

# import gc

# import numpy as np
# import pandas as pd
# from pathlib import Path

# from pandarallel import pandarallel
# pandarallel.initialize()

# BASE_DIR = Path('../input/mlb-player-digital-engagement-forecasting')
# train = pd.read_csv(BASE_DIR / 'train.csv')

# null = np.nan
# true = True
# false = False

# for col in train.columns:

#     if col == 'date': continue

#     _index = train[col].notnull()
#     train.loc[_index, col] = train.loc[_index, col].parallel_apply(lambda x: eval(x))

#     outputs = []
#     for index, date, record in train.loc[_index, ['date', col]].itertuples():
#         _df = pd.DataFrame(record)
#         _df['index'] = index
#         _df['date'] = date
#         outputs.append(_df)

#     outputs = pd.concat(outputs).reset_index(drop=True)

#     outputs.to_csv(f'{col}_train.csv', index=False)
#     outputs.to_pickle(f'{col}_train.pkl')

#     del outputs
#     del train[col]
#     gc.collect()

##### Checking / Collecting our data files  

In [80]:
path_target = "/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/nextDayPlayerEngagement_train.pkl"
targets = pd.read_pickle(path)

In [22]:
display(targets)

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date
0,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294,0,20180101
1,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118,0,20180101
2,2018-01-02,519317,0.974327,56.177043,13.693746,64.166667,0,20180101
3,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745,0,20180101
4,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373,0,20180101
...,...,...,...,...,...,...,...,...
2506171,2021-05-01,451661,0.000000,0.013314,0.000000,0.625925,1215,20210430
2506172,2021-05-01,519301,0.000131,0.003329,0.000000,0.216229,1215,20210430
2506173,2021-05-01,527055,0.000000,0.019971,0.000000,0.273131,1215,20210430
2506174,2021-05-01,543484,0.000131,0.056586,0.000000,1.024240,1215,20210430


In [82]:
path_bs = '/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/playerBoxScores_train.pkl'
playerBoxScores_train = pd.read_pickle(path_bs)

In [83]:
path_players = '/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/players.csv'
players = pd.read_csv(path_players)

rosters_path = '/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/rosters_train.pkl'
rosters = pd.read_pickle(rosters_path)



##### Setting up data for NN 

In [25]:
pl_eng_w_scores = pd.merge(targets,playerBoxScores_train,on=['date','playerId'],how = 'inner')
display(pl_eng_w_scores)

,engagementMetricsDate,playerId,target1,target2,target3,target4,index_x,date,home,gamePk,...,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,index_y
0,2018-03-30,608365,2.040838,7.893376,0.052359,2.599964,87,20180329,0,529407,...,NaN,NaN,NaN,NaN,NaN,5.0,3.0,1.0,9.0,87
1,2018-03-30,502624,1.068945,5.105543,2.722191,0.918550,87,20180329,0,529413,...,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,87
2,2018-03-30,643338,0.385945,5.368812,0.074404,0.565661,87,20180329,0,529415,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,87
3,2018-03-30,570560,0.131860,6.200931,0.002296,0.181634,87,20180329,0,529408,...,NaN,NaN,NaN,NaN,NaN,4.0,1.0,0.0,5.0,87
4,2018-03-30,523260,0.011800,9.933713,6.076820,0.978230,87,20180329,0,529406,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184754,2021-05-01,664057,0.002498,1.963852,0.000000,1.285991,1215,20210430,1,634274,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1215
184755,2021-05-01,664854,0.446403,0.625770,0.000000,0.273131,1215,20210430,0,634325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1215
184756,2021-05-01,605612,0.016699,1.198282,0.000000,0.739729,1215,20210430,0,634327,...,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,2.0,1215
184757,2021-05-01,643778,0.000000,0.595813,0.000000,0.273131,1215,20210430,0,634325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1215


In [26]:
# we want to get birthCountry, primaryPositionName, rosterTeamId, rosterStatus for each player

players_cols = ['playerId', 'primaryPositionName', 'birthCountry']
rosters_cols = ['playerId', 'teamId', 'status', 'date']

position_birth = players[players_cols]
rosters_small = rosters[rosters_cols]
categorical_info = pd.merge(rosters_small,position_birth,on=['playerId'],how = 'left')

pl_eng_w_scores_c = pd.merge(pl_eng_w_scores,categorical_info,on=['playerId' , 'date' ,'teamId'],how = 'left').reset_index()

display(pl_eng_w_scores_c)

,index,engagementMetricsDate,playerId,target1,target2,target3,target4,index_x,date,home,...,holds,blownSaves,assists,putOuts,errors,chances,index_y,status,primaryPositionName,birthCountry
0,0,2018-03-30,608365,2.040838,7.893376,0.052359,2.599964,87,20180329,0,...,NaN,NaN,5.0,3.0,1.0,9.0,87,Active,Second Base,USA
1,1,2018-03-30,502624,1.068945,5.105543,2.722191,0.918550,87,20180329,0,...,NaN,NaN,0.0,0.0,0.0,0.0,87,Active,Pitcher,USA
2,2,2018-03-30,643338,0.385945,5.368812,0.074404,0.565661,87,20180329,0,...,0.0,0.0,0.0,1.0,0.0,1.0,87,Active,Pitcher,USA
3,3,2018-03-30,570560,0.131860,6.200931,0.002296,0.181634,87,20180329,0,...,NaN,NaN,4.0,1.0,0.0,5.0,87,Active,Second Base,Venezuela
4,4,2018-03-30,523260,0.011800,9.933713,6.076820,0.978230,87,20180329,0,...,0.0,0.0,NaN,NaN,NaN,NaN,87,Active,Pitcher,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184754,184754,2021-05-01,664057,0.002498,1.963852,0.000000,1.285991,1215,20210430,1,...,NaN,NaN,0.0,0.0,0.0,0.0,1215,Active,Outfielder,USA
184755,184755,2021-05-01,664854,0.446403,0.625770,0.000000,0.273131,1215,20210430,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1215,Active,Pitcher,USA
184756,184756,2021-05-01,605612,0.016699,1.198282,0.000000,0.739729,1215,20210430,0,...,NaN,NaN,0.0,2.0,0.0,2.0,1215,Active,Second Base,Venezuela
184757,184757,2021-05-01,643778,0.000000,0.595813,0.000000,0.273131,1215,20210430,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1215,Active,Pitcher,USA


In [27]:
del pl_eng_w_scores_c

In [28]:
display(targets)
#resetting 
#targets = pd.read_pickle("/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/nextDayPlayerEngagement_train.pkl")

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date
0,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294,0,20180101
1,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118,0,20180101
2,2018-01-02,519317,0.974327,56.177043,13.693746,64.166667,0,20180101
3,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745,0,20180101
4,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373,0,20180101
...,...,...,...,...,...,...,...,...
2506171,2021-05-01,451661,0.000000,0.013314,0.000000,0.625925,1215,20210430
2506172,2021-05-01,519301,0.000131,0.003329,0.000000,0.216229,1215,20210430
2506173,2021-05-01,527055,0.000000,0.019971,0.000000,0.273131,1215,20210430
2506174,2021-05-01,543484,0.000131,0.056586,0.000000,1.024240,1215,20210430


### Autoregressive Neural Network - 5 lags 

#### Base Model 
- we build baseline model first , i.e. run all the cells until hyper paramter tuning , to get a 'feel' for what the model would be 
- then we tune hyper parameters 

In [102]:
#targets = pd.read_pickle("/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/nextDayPlayerEngagement_train.pkl")

In [3]:
def reset():
    path_target = "/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/nextDayPlayerEngagement_train.pkl"
    targets = pd.read_pickle(path_target)
    #data differencing 
    testing = targets["target1"]
    diff = testing.diff()
    targets["t1_diff"] = diff

    testing = targets["target2"]
    diff = testing.diff()
    targets["t2_diff"] = diff

    testing = targets["target3"]
    diff = testing.diff()
    targets["t3_diff"] = diff

    testing = targets["target4"]
    diff = testing.diff()
    targets["t4_diff"] = diff
    
    return targets 

targets = reset()
display(targets)

#### we now run the code cells in order and build a base line model , which we later tune 

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,t3_diff,t4_diff
0,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294,0,20180101,NaN,NaN,NaN,NaN
1,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118,0,20180101,0.031826,1.118677,0.039865,-2.941176
2,2018-01-02,519317,0.974327,56.177043,13.693746,64.166667,0,20180101,0.931334,50.583658,13.648712,61.372549
3,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745,0,20180101,-0.967627,-53.501946,-13.688578,-62.303922
4,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373,0,20180101,-0.005584,-2.042802,-0.002215,-0.931373
...,...,...,...,...,...,...,...,...,...,...,...,...
2506171,2021-05-01,451661,0.000000,0.013314,0.000000,0.625925,1215,20210430,0.000000,-0.003329,0.000000,0.534881
2506172,2021-05-01,519301,0.000131,0.003329,0.000000,0.216229,1215,20210430,0.000131,-0.009986,0.000000,-0.409696
2506173,2021-05-01,527055,0.000000,0.019971,0.000000,0.273131,1215,20210430,-0.000131,0.016643,0.000000,0.056902
2506174,2021-05-01,543484,0.000131,0.056586,0.000000,1.024240,1215,20210430,0.000131,0.036614,0.000000,0.751110


In [4]:
from datetime import timedelta

In [5]:
trgt_lags = targets
#display(trgt_lags)

In [6]:
# Functions below will iteratively adds lagged version of targets to the feature set we build. 

In [7]:
# using the next day prediction targets as the features in our model by changing the date to one day back 
# then we can build lags using this 
def day_back(trgt_lags):
    trgt_lags["engagementMetricsDate"] = pd.to_datetime(trgt_lags["engagementMetricsDate"])
    trgt_lags["engagementMetricsDate"] = trgt_lags["engagementMetricsDate"] + timedelta(days=-1)

In [8]:
day_back(trgt_lags)

In [9]:
display(trgt_lags) #checking if our dates did get pushed back 

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,t3_diff,t4_diff
0,2018-01-01,628317,0.011167,4.474708,0.005168,5.735294,0,20180101,NaN,NaN,NaN,NaN
1,2018-01-01,547989,0.042993,5.593385,0.045033,2.794118,0,20180101,0.031826,1.118677,0.039865,-2.941176
2,2018-01-01,519317,0.974327,56.177043,13.693746,64.166667,0,20180101,0.931334,50.583658,13.648712,61.372549
3,2018-01-01,607625,0.006700,2.675097,0.005168,1.862745,0,20180101,-0.967627,-53.501946,-13.688578,-62.303922
4,2018-01-01,592547,0.001117,0.632296,0.002953,0.931373,0,20180101,-0.005584,-2.042802,-0.002215,-0.931373
...,...,...,...,...,...,...,...,...,...,...,...,...
2506171,2021-04-30,451661,0.000000,0.013314,0.000000,0.625925,1215,20210430,0.000000,-0.003329,0.000000,0.534881
2506172,2021-04-30,519301,0.000131,0.003329,0.000000,0.216229,1215,20210430,0.000131,-0.009986,0.000000,-0.409696
2506173,2021-04-30,527055,0.000000,0.019971,0.000000,0.273131,1215,20210430,-0.000131,0.016643,0.000000,0.056902
2506174,2021-04-30,543484,0.000131,0.056586,0.000000,1.024240,1215,20210430,0.000131,0.036614,0.000000,0.751110


In [10]:
trgt_cols = ["t1_diff","t2_diff","t3_diff","t4_diff"]

In [11]:
def train_lag(df, lag):
    #get copy of current data frame , with targets values 
    dp = df[["playerId","engagementMetricsDate"]+trgt_cols].copy()
    # set date to 'lag' many days AHEAD 
    dp["engagementMetricsDate"]  = dp["engagementMetricsDate"] + timedelta(days=lag) 
    # now when you merge the data frame dp on to the original data frame youll be creating features  that are lagged values 
    df = df.merge(dp, on=["playerId", "engagementMetricsDate"], suffixes=["",f"_{lag}"], how="left")
    return df

**If anything is changed in the code you must re intialise the trgt_lags variable and consequently train_lag**

In [12]:
#building our feature set ...

def create_lags(max_lag, offset, trgt_lags):
    MAX_LAG = max_lag 
    OFFSET = offset
    # get list of lags 
    LAGS = list(range(OFFSET, MAX_LAG + OFFSET))
    
    # naming our features 
    FECOLS = [f"{col}_{lag}" for lag in reversed(LAGS) for col in trgt_cols]
    
    # iteratively building our feature set 
    for lag in LAGS:
        trgt_lags = train_lag(trgt_lags, lag=lag)
        #print('train---')
        #display(trgt_lags)
        gc.collect()
    return trgt_lags, FECOLS

# max_lag found from data exploration , but using trail and error MAX_LAG of 20 produced the best results 
# offset is how many days before you want to start the lags 

In [13]:
%%time 
#creating our lags data set 
def create_lags_dataset(trgt_lags , LAG, OFFSET):
    trgt_lags, FECOLS = create_lags(LAG, OFFSET, trgt_lags)
    trgt_lags = trgt_lags.sort_values(by=["playerId", "engagementMetricsDate"])
    print('train sorted---')
    display(trgt_lags)
    trgt_lags = trgt_lags.dropna()
    print(trgt_lags.shape)
    gc.collect()
    
    return trgt_lags, FECOLS

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 11 µs


In [14]:
trgt_lags, FECOLS = create_lags_dataset(trgt_lags , 5, 1)

train sorted---


,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,...,t3_diff_3,t4_diff_3,t1_diff_4,t2_diff_4,t3_diff_4,t4_diff_4,t1_diff_5,t2_diff_5,t3_diff_5,t4_diff_5
969,2018-01-01,112526,0.055277,5.496109,0.025839,16.176471,0,20180101,0.052485,2.383268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3556,2018-01-02,112526,0.060625,3.252914,0.030486,8.541353,1,20180102,0.052314,1.789103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5737,2018-01-03,112526,0.029341,1.648352,0.032613,10.490112,2,20180103,0.029341,1.579670,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7768,2018-01-04,112526,0.014799,2.665894,0.087422,19.091467,3,20180104,0.014799,1.912489,...,0.025100,15.686275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8963,2018-01-05,112526,0.083916,1.161002,0.024759,6.643880,4,20180105,0.082203,0.363687,...,-0.004120,7.969925,0.052485,2.383268,0.025100,15.686275,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495980,2021-04-26,685503,0.044617,1.224728,0.009437,0.737463,1211,20210426,0.018820,-0.018463,...,-0.051184,-2.655033,0.099302,-0.373574,0.007259,-1.477378,0.121678,1.688486,0.157717,-0.092081
2497962,2021-04-27,685503,0.019123,1.178880,0.013161,0.790301,1212,20210427,0.017971,-0.950709,...,-1.471642,-0.055074,-0.062143,-3.985773,-0.051184,-2.655033,0.099302,-0.373574,0.007259,-1.477378
2500311,2021-04-28,685503,0.015799,4.323489,0.002350,0.970273,1213,20210428,-0.273695,0.741017,...,0.014520,-0.006554,-0.063314,14.899376,-1.471642,-0.055074,-0.062143,-3.985773,-0.051184,-2.655033
2502400,2021-04-29,685503,0.018770,31.946021,0.305491,5.938273,1214,20210429,0.011956,31.854310,...,-0.052953,-0.120676,0.075090,1.644315,0.014520,-0.006554,-0.063314,14.899376,-1.471642,-0.055074


(2495870, 32)


In [15]:
display(trgt_lags.columns)

Index(['engagementMetricsDate', 'playerId', 'target1', 'target2', 'target3',
       'target4', 'index', 'date', 't1_diff', 't2_diff', 't3_diff', 't4_diff',
       't1_diff_1', 't2_diff_1', 't3_diff_1', 't4_diff_1', 't1_diff_2',
       't2_diff_2', 't3_diff_2', 't4_diff_2', 't1_diff_3', 't2_diff_3',
       't3_diff_3', 't4_diff_3', 't1_diff_4', 't2_diff_4', 't3_diff_4',
       't4_diff_4', 't1_diff_5', 't2_diff_5', 't3_diff_5', 't4_diff_5'],
      dtype='object')

In [16]:
def create_train_test(trgt_lags): 
    trgt_lags.fillna(-1,inplace = True)

    sample_y = trgt_lags[['t1_diff','t2_diff', 't3_diff', 't4_diff']]
    sample_y = sample_y.reset_index(drop=True)
    sample_X = trgt_lags[FECOLS]
    sample_X = sample_X.reset_index(drop=True)

    # display(sample_X , sample_X.columns)
    # display(sample_y)


    from sklearn.model_selection import train_test_split


    X_train, X_test, y_train, y_test = train_test_split(sample_X, sample_y, test_size=0.2, shuffle=False)
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    
    return X_train, X_test, y_train, y_test

In [17]:
X_train, X_test, y_train, y_test = create_train_test(trgt_lags)


In [18]:
display(X_train , y_train)

,t1_diff_5,t2_diff_5,t3_diff_5,t4_diff_5,t1_diff_4,t2_diff_4,t3_diff_4,t4_diff_4,t1_diff_3,t2_diff_3,t3_diff_3,t4_diff_3,t1_diff_2,t2_diff_2,t3_diff_2,t4_diff_2,t1_diff_1,t2_diff_1,t3_diff_1,t4_diff_1
0,0.052485,2.383268,0.025100,15.686275,0.052314,1.789103,-0.004120,7.969925,0.029341,1.579670,0.031707,10.361135,0.014799,1.912489,0.083324,18.477594,0.082203,0.363687,-0.035263,6.451613
1,0.052314,1.789103,-0.004120,7.969925,0.029341,1.579670,0.031707,10.361135,0.014799,1.912489,0.083324,18.477594,0.082203,0.363687,-0.035263,6.451613,0.020096,1.664499,0.031718,11.976048
2,0.029341,1.579670,0.031707,10.361135,0.014799,1.912489,0.083324,18.477594,0.082203,0.363687,-0.035263,6.451613,0.020096,1.664499,0.031718,11.976048,0.014554,3.314213,0.006584,15.866873
3,0.014799,1.912489,0.083324,18.477594,0.082203,0.363687,-0.035263,6.451613,0.020096,1.664499,0.031718,11.976048,0.014554,3.314213,0.006584,15.866873,0.032364,1.788376,0.034842,17.460317
4,0.082203,0.363687,-0.035263,6.451613,0.020096,1.664499,0.031718,11.976048,0.014554,3.314213,0.006584,15.866873,0.032364,1.788376,0.034842,17.460317,0.016104,-1.028192,0.049261,4.262352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996691,-1.396342,-3.998197,0.002610,-0.561379,-0.230766,-7.299600,-0.056443,-2.933952,-0.003617,0.867656,-0.004022,-0.177069,-0.057194,-0.231512,0.007180,-0.085772,-0.023527,-0.056529,-0.767862,-0.100511
1996692,-0.230766,-7.299600,-0.056443,-2.933952,-0.003617,0.867656,-0.004022,-0.177069,-0.057194,-0.231512,0.007180,-0.085772,-0.023527,-0.056529,-0.767862,-0.100511,-0.000413,0.071544,0.007848,-0.272639
1996693,-0.003617,0.867656,-0.004022,-0.177069,-0.057194,-0.231512,0.007180,-0.085772,-0.023527,-0.056529,-0.767862,-0.100511,-0.000413,0.071544,0.007848,-0.272639,-1.274135,-3.522829,-13.303048,-11.986363
1996694,-0.057194,-0.231512,0.007180,-0.085772,-0.023527,-0.056529,-0.767862,-0.100511,-0.000413,0.071544,0.007848,-0.272639,-1.274135,-3.522829,-13.303048,-11.986363,-0.000668,0.688557,0.000716,-0.073359


,t1_diff,t2_diff,t3_diff,t4_diff
0,0.020096,1.664499,0.031718,11.976048
1,0.014554,3.314213,0.006584,15.866873
2,0.032364,1.788376,0.034842,17.460317
3,0.016104,-1.028192,0.049261,4.262352
4,0.019394,0.549089,-0.102723,8.506344
...,...,...,...,...
1996691,-0.000413,0.071544,0.007848,-0.272639
1996692,-1.274135,-3.522829,-13.303048,-11.986363
1996693,-0.000668,0.688557,0.000716,-0.073359
1996694,-0.001251,0.077825,0.000738,-0.082282


In [19]:
# dropout in hidden layers with weight constraint
def create_model(input_dim , HIDDEN , ACTIVATION, DROPOUT_RATE,LEARNING_RATE):
    #create model 
    model = keras.Sequential([
        #keras.Input(shape=(input_dim,)),
        layers.Dense(HIDDEN, activation=ACTIVATION),
        layers.BatchNormalization(),
        layers.Dropout(DROPOUT_RATE),
        layers.Dense(HIDDEN, activation=ACTIVATION),
        layers.BatchNormalization(),
        layers.Dropout(DROPOUT_RATE),
        layers.Dense(4),#output
    ])
    # Compile model
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])
    return model

##### Training nueral network with Cross validation 

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

def run_ARNN(X_train, X_test, y_train, y_test, hidden, dropuout_rate, learning_rate):
    # Define the model architecture
    HIDDEN = hidden
    ACTIVATION = 'relu'  
    DROPOUT_RATE = dropuout_rate
    LEARNING_RATE = learning_rate
    BATCH_SIZE = 5000 # increased batch size as 2.5mill+ rows 
    OUTPUTS = 4

    # Define per-fold score containers
    acc_per_fold = []
    loss_per_fold = []
    
    # Define the K-fold Cross Validator
    tscv = TimeSeriesSplit(n_splits=10)

    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train_index, val_index in tscv.split(X_train):
        input_dim = X_train.shape[0]
        model = create_model(input_dim , HIDDEN , ACTIVATION, DROPOUT_RATE,LEARNING_RATE)
        # Generate a print
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        history = model.fit(
        X_train.iloc[train_index], y_train.iloc[train_index], #validation_data=(X_valid, y_valid)
        batch_size=BATCH_SIZE,
        epochs=15,
        verbose=1)


        # Generate generalization metrics
        scores = model.evaluate(X_train.iloc[val_index], y_train.iloc[val_index], verbose=0)
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])

        # Increase fold number
        fold_no = fold_no + 1
    #after training we evaulte on test set 
    
        
    return acc_per_fold, loss_per_fold, model

In [ ]:


# Hyperparameters
# HIDDEN = 1024
# ACTIVATION = 'relu'  # could try elu, gelu, swish
# DROPOUT_RATE = 0.5
# LEARNING_RATE = 1e-2
# BATCH_SIZE = 32


acc_per_fold, loss_per_fold , model = run_ARNN(X_train, X_test, y_train, y_test, 50, 0.2, 1e-2)



------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15
37/37 [==============================] - 2s 11ms/step - loss: 2.9500 - mae: 2.9500
Epoch 2/15
37/37 [==============================] - 0s 10ms/step - loss: 2.8016 - mae: 2.8016
Epoch 3/15
37/37 [==============================] - 0s 9ms/step - loss: 2.7831 - mae: 2.7831
Epoch 4/15
37/37 [==============================] - 0s 10ms/step - loss: 2.7674 - mae: 2.7674
Epoch 5/15
37/37 [==============================] - 0s 9ms/step - loss: 2.7607 - mae: 2.7607
Epoch 6/15
37/37 [==============================] - 0s 9ms/step - loss: 2.7571 - mae: 2.7571
Epoch 7/15
37/37 [==============================] - 0s 10ms/step - loss: 2.7544 - mae: 2.7544
Epoch 8/15
37/37 [==============================] - 0s 11ms/step - loss: 2.7514 - mae: 2.7514
Epoch 9/15
37/37 [==============================] - 0s 10ms/step - loss: 2.7485 - mae: 2.7485
Epoch 10/15
37/37 [==============================] - 0s 11ms

In [53]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
    
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [32]:
#results on test set
predictions = model.predict(X_test)
display(predictions)
scores = model.evaluate(X_test, y_test, verbose=0)
print(f'Score for test {model.metrics_names[0]} of {scores[0]}')

In [125]:
#targets = pd.read_pickle("/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/nextDayPlayerEngagement_train.pkl")

#### Hyper paramter methodology 

- Once baseline model has been run 
- we go through each tuning section dropout , nodes and learning rate 
    - once you have run one section update the base line function, by updating the relevant variable with the best results obtained 
    - base line function : 
       - run_ARNN(X_train, X_test, y_train, y_test, hidden, dropuout_rate, learning_rate):
       - variables to update inside function 
       - HIDDEN = hidden
       - DROPOUT_RATE = dropuout_rate
       - LEARNING_RATE = learning_rate
- hence you will iteratively building the best model 
- we can then go through and build the rest of the models with these optimised paramters (setting the varaibles in the run_ARNN function) as its the same model just with slightly different data 
- this method was chose to minimize training time and still give adequt results as opposed to keras tunner 

### Hyper paramter tuning with 5 lags 

#### Testing different dropout layers 

In [61]:
del trgt_lags, X_train, X_test, y_train, y_test 

In [88]:
x_dp = []
y_dp = []
LAG = 5 
OFFSET = 1
def get_best_dp():
    for dp in np.arange(0, 1.0, 0.1):
        print(f"hidden is --- {dp}")

        #get targets
        targets = reset()
        trgt_lags = targets
        day_back(trgt_lags)

        #get target lags data set 
        trgt_lags, FECOLS = create_lags_dataset(trgt_lags ,LAG ,OFFSET)

        display(trgt_lags)
        #get sample set 
        X_train, X_test, y_train, y_test = create_train_test(trgt_lags)

        #get model results acc_per_fold, loss_per_fold , model                           hidden, dropuout_rate, learning_rate
        acc_per_fold, loss_per_fold , model = run_ARNN(X_train, X_test, y_train, y_test, 50, dp, 1e-2)

        #find avg loss from each fold 
        loss = np.mean(loss_per_fold)
        #append to arrays 
        x_dp.append(dp)
        y_dp.append(round(loss, 3))

        #delete all data sets to reset  
        del trgt_lags, X_train, X_test, y_train, y_test, acc_per_fold, loss_per_fold , model


In [ ]:
#get_best_dp() -- run this to get the arrays
index = y_dp.index(min(y_dp))
best_dp = x_dp[index]
print(best_dp)

In [124]:
del trgt_lags, X_train, X_test, y_train, y_test, acc_per_fold, loss_per_fold , model

#### Testing number of Hidden Nodes

In [89]:
x_hid = []
y_hid = []

LAG = 5 
OFFSET = 1
# train first with 2 
def get_best_hid():
    for hid in np.arange(50, 120, 10):
        print(f"hidden is --- {hid}")

        #get targets
        targets = reset()
        trgt_lags = targets
        day_back(trgt_lags)

        #get target lags data set 
        trgt_lags, FECOLS = create_lags_dataset(trgt_lags, LAG, OFFSET)

        display(trgt_lags)
        #get sample set 
        X_train, X_test, y_train, y_test = create_train_test(trgt_lags)

        #get model results acc_per_fold, loss_per_fold , model                           hidden, dropuout_rate, learning_rate
        acc_per_fold, loss_per_fold , model = run_ARNN(X_train, X_test, y_train, y_test, hid, 0.2, 1e-2)

        #find avg loss from each fold 
        loss = np.mean(loss_per_fold)
        #append to arrays 
        x_hid.append(hid)
        y_hid.append(round(loss, 3))

        #delete all data sets to reset  
        del trgt_lags, X_train, X_test, y_train, y_test, acc_per_fold, loss_per_fold , model

#get minmum 

In [123]:
get_best_hid()
index = y_hid.index(min(y_hid))
best_hid = x_hid[index]
print(best_hid)

#### Testing different of learning rates 

In [101]:
x_lr = []
y_lr = []

LAG = 5 
OFFSET = 1
# train first with 2 
def get_best_lr():
    for lr in np.arange(0.001, 0.11, 0.02):
        print(f"lr is --- {lr}")

        #get targets
        targets = reset()
        trgt_lags = targets
        day_back(trgt_lags)

        #get target lags data set 
        trgt_lags, FECOLS = create_lags_dataset(trgt_lags, LAG, OFFSET)

        display(trgt_lags)
        #get sample set 
        X_train, X_test, y_train, y_test = create_train_test(trgt_lags)

        #get model results acc_per_fold, loss_per_fold , model                           hidden, dropuout_rate, learning_rate
        acc_per_fold, loss_per_fold , model = run_ARNN(X_train, X_test, y_train, y_test, 50, 0.2, lr)

        #find avg loss from each fold 
        loss = np.mean(loss_per_fold)
        #append to arrays 
        x_lr.append(lr)
        y_lr.append(round(loss, 3))

        #delete all data sets to reset  
        del trgt_lags, X_train, X_test, y_train, y_test, acc_per_fold, loss_per_fold , model



In [122]:
get_best_lr()
index = y_lr.index(min(y_lr))
best_lr = x_lr[index]
print(best_lr)

0.001


### Auto Regressive nueral network 20 lags 

In [ ]:
# from the best hyper parameters above we re train the models , with those parameters 

In [119]:
#building the data again but with 20 lags 
targets = reset()
# collecting what we need 
trgt_lags = targets
day_back(trgt_lags)

In [120]:
display(trgt_lags)

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,t3_diff,t4_diff
0,2018-01-01,628317,0.011167,4.474708,0.005168,5.735294,0,20180101,NaN,NaN,NaN,NaN
1,2018-01-01,547989,0.042993,5.593385,0.045033,2.794118,0,20180101,0.031826,1.118677,0.039865,-2.941176
2,2018-01-01,519317,0.974327,56.177043,13.693746,64.166667,0,20180101,0.931334,50.583658,13.648712,61.372549
3,2018-01-01,607625,0.006700,2.675097,0.005168,1.862745,0,20180101,-0.967627,-53.501946,-13.688578,-62.303922
4,2018-01-01,592547,0.001117,0.632296,0.002953,0.931373,0,20180101,-0.005584,-2.042802,-0.002215,-0.931373
...,...,...,...,...,...,...,...,...,...,...,...,...
2506171,2021-04-30,451661,0.000000,0.013314,0.000000,0.625925,1215,20210430,0.000000,-0.003329,0.000000,0.534881
2506172,2021-04-30,519301,0.000131,0.003329,0.000000,0.216229,1215,20210430,0.000131,-0.009986,0.000000,-0.409696
2506173,2021-04-30,527055,0.000000,0.019971,0.000000,0.273131,1215,20210430,-0.000131,0.016643,0.000000,0.056902
2506174,2021-04-30,543484,0.000131,0.056586,0.000000,1.024240,1215,20210430,0.000131,0.036614,0.000000,0.751110


In [121]:
%%time
#building our feature set ...
lags = 20 
offset = 45 
trgt_lags, FECOLS = create_lags_dataset(trgt_lags , lags, offset)
trgt_lags = trgt_lags.sort_values(by=["playerId", "engagementMetricsDate"])
print('train sorted---')
display(trgt_lags)
trgt_lags = trgt_lags.dropna()
print(trgt_lags.shape)
gc.collect()


train sorted---


,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,...,t3_diff_62,t4_diff_62,t1_diff_63,t2_diff_63,t3_diff_63,t4_diff_63,t1_diff_64,t2_diff_64,t3_diff_64,t4_diff_64
969,2018-01-01,112526,0.055277,5.496109,0.025839,16.176471,0,20180101,0.052485,2.383268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3556,2018-01-02,112526,0.060625,3.252914,0.030486,8.541353,1,20180102,0.052314,1.789103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5737,2018-01-03,112526,0.029341,1.648352,0.032613,10.490112,2,20180103,0.029341,1.579670,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7768,2018-01-04,112526,0.014799,2.665894,0.087422,19.091467,3,20180104,0.014799,1.912489,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8963,2018-01-05,112526,0.083916,1.161002,0.024759,6.643880,4,20180105,0.082203,0.363687,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495980,2021-04-26,685503,0.044617,1.224728,0.009437,0.737463,1211,20210426,0.018820,-0.018463,...,0.178820,1.381861,-0.002389,2.189266,0.550148,-0.351083,-0.213468,1.186673,1.000105,-0.177141
2497962,2021-04-27,685503,0.019123,1.178880,0.013161,0.790301,1212,20210427,0.017971,-0.950709,...,-0.067067,-7.507127,-0.002370,4.621514,0.178820,1.381861,-0.002389,2.189266,0.550148,-0.351083
2500311,2021-04-28,685503,0.015799,4.323489,0.002350,0.970273,1213,20210428,-0.273695,0.741017,...,0.064103,1.355276,-0.089255,-2.664129,-0.067067,-7.507127,-0.002370,4.621514,0.178820,1.381861
2502400,2021-04-29,685503,0.018770,31.946021,0.305491,5.938273,1214,20210429,0.011956,31.854310,...,0.017022,-0.429676,0.000000,3.394372,0.064103,1.355276,-0.089255,-2.664129,-0.067067,-7.507127


(2374271, 92)
train sorted---


,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,...,t3_diff_62,t4_diff_62,t1_diff_63,t2_diff_63,t3_diff_63,t4_diff_63,t1_diff_64,t2_diff_64,t3_diff_64,t4_diff_64
132208,2018-03-06,112526,0.189106,6.780566,2.209322,2.322992,64,20180306,0.181572,3.230112,...,0.031707,10.361135,0.052314,1.789103,-0.004120,7.969925,0.052485,2.383268,0.025100,15.686275
135698,2018-03-07,112526,0.053513,4.867257,3.300729,2.300283,65,20180307,0.040965,4.582807,...,0.083324,18.477594,0.029341,1.579670,0.031707,10.361135,0.052314,1.789103,-0.004120,7.969925
137091,2018-03-08,112526,0.066996,11.750517,3.492547,7.134919,66,20180308,0.063152,4.376292,...,-0.035263,6.451613,0.014799,1.912489,0.083324,18.477594,0.029341,1.579670,0.031707,10.361135
139104,2018-03-09,112526,0.249823,18.964578,1.259076,9.912340,67,20180309,0.245613,11.253406,...,0.031718,11.976048,0.082203,0.363687,-0.035263,6.451613,0.014799,1.912489,0.083324,18.477594
141554,2018-03-10,112526,1.302126,56.259052,1.190449,10.539515,68,20180310,1.282909,54.769294,...,0.006584,15.866873,0.020096,1.664499,0.031718,11.976048,0.082203,0.363687,-0.035263,6.451613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495980,2021-04-26,685503,0.044617,1.224728,0.009437,0.737463,1211,20210426,0.018820,-0.018463,...,0.178820,1.381861,-0.002389,2.189266,0.550148,-0.351083,-0.213468,1.186673,1.000105,-0.177141
2497962,2021-04-27,685503,0.019123,1.178880,0.013161,0.790301,1212,20210427,0.017971,-0.950709,...,-0.067067,-7.507127,-0.002370,4.621514,0.178820,1.381861,-0.002389,2.189266,0.550148,-0.351083
2500311,2021-04-28,685503,0.015799,4.323489,0.002350,0.970273,1213,20210428,-0.273695,0.741017,...,0.064103,1.355276,-0.089255,-2.664129,-0.067067,-7.507127,-0.002370,4.621514,0.178820,1.381861
2502400,2021-04-29,685503,0.018770,31.946021,0.305491,5.938273,1214,20210429,0.011956,31.854310,...,0.017022,-0.429676,0.000000,3.394372,0.064103,1.355276,-0.089255,-2.664129,-0.067067,-7.507127


(2374271, 92)
CPU times: user 1min 1s, sys: 1min 17s, total: 2min 18s
Wall time: 3min 15s


0

In [125]:
display(trgt_lags, trgt_lags.columns)

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,...,t3_diff_62,t4_diff_62,t1_diff_63,t2_diff_63,t3_diff_63,t4_diff_63,t1_diff_64,t2_diff_64,t3_diff_64,t4_diff_64
132208,2018-03-06,112526,0.189106,6.780566,2.209322,2.322992,64,20180306,0.181572,3.230112,...,0.031707,10.361135,0.052314,1.789103,-0.004120,7.969925,0.052485,2.383268,0.025100,15.686275
135698,2018-03-07,112526,0.053513,4.867257,3.300729,2.300283,65,20180307,0.040965,4.582807,...,0.083324,18.477594,0.029341,1.579670,0.031707,10.361135,0.052314,1.789103,-0.004120,7.969925
137091,2018-03-08,112526,0.066996,11.750517,3.492547,7.134919,66,20180308,0.063152,4.376292,...,-0.035263,6.451613,0.014799,1.912489,0.083324,18.477594,0.029341,1.579670,0.031707,10.361135
139104,2018-03-09,112526,0.249823,18.964578,1.259076,9.912340,67,20180309,0.245613,11.253406,...,0.031718,11.976048,0.082203,0.363687,-0.035263,6.451613,0.014799,1.912489,0.083324,18.477594
141554,2018-03-10,112526,1.302126,56.259052,1.190449,10.539515,68,20180310,1.282909,54.769294,...,0.006584,15.866873,0.020096,1.664499,0.031718,11.976048,0.082203,0.363687,-0.035263,6.451613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495980,2021-04-26,685503,0.044617,1.224728,0.009437,0.737463,1211,20210426,0.018820,-0.018463,...,0.178820,1.381861,-0.002389,2.189266,0.550148,-0.351083,-0.213468,1.186673,1.000105,-0.177141
2497962,2021-04-27,685503,0.019123,1.178880,0.013161,0.790301,1212,20210427,0.017971,-0.950709,...,-0.067067,-7.507127,-0.002370,4.621514,0.178820,1.381861,-0.002389,2.189266,0.550148,-0.351083
2500311,2021-04-28,685503,0.015799,4.323489,0.002350,0.970273,1213,20210428,-0.273695,0.741017,...,0.064103,1.355276,-0.089255,-2.664129,-0.067067,-7.507127,-0.002370,4.621514,0.178820,1.381861
2502400,2021-04-29,685503,0.018770,31.946021,0.305491,5.938273,1214,20210429,0.011956,31.854310,...,0.017022,-0.429676,0.000000,3.394372,0.064103,1.355276,-0.089255,-2.664129,-0.067067,-7.507127


Index(['engagementMetricsDate', 'playerId', 'target1', 'target2', 'target3',
       'target4', 'index', 'date', 't1_diff', 't2_diff', 't3_diff', 't4_diff',
       't1_diff_45', 't2_diff_45', 't3_diff_45', 't4_diff_45', 't1_diff_46',
       't2_diff_46', 't3_diff_46', 't4_diff_46', 't1_diff_47', 't2_diff_47',
       't3_diff_47', 't4_diff_47', 't1_diff_48', 't2_diff_48', 't3_diff_48',
       't4_diff_48', 't1_diff_49', 't2_diff_49', 't3_diff_49', 't4_diff_49',
       't1_diff_50', 't2_diff_50', 't3_diff_50', 't4_diff_50', 't1_diff_51',
       't2_diff_51', 't3_diff_51', 't4_diff_51', 't1_diff_52', 't2_diff_52',
       't3_diff_52', 't4_diff_52', 't1_diff_53', 't2_diff_53', 't3_diff_53',
       't4_diff_53', 't1_diff_54', 't2_diff_54', 't3_diff_54', 't4_diff_54',
       't1_diff_55', 't2_diff_55', 't3_diff_55', 't4_diff_55', 't1_diff_56',
       't2_diff_56', 't3_diff_56', 't4_diff_56', 't1_diff_57', 't2_diff_57',
       't3_diff_57', 't4_diff_57', 't1_diff_58', 't2_diff_58', 't3_diff_58

In [61]:
trgt_lags.fillna(-1,inplace = True)

sample_y = trgt_lags[['t1_diff','t2_diff', 't3_diff', 't4_diff']]
sample_y = sample_y.reset_index(drop=True)
sample_X = trgt_lags[FECOLS]
sample_X = sample_X.reset_index(drop=True)

display(sample_X , sample_X.columns)
display(sample_y)


from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(sample_X, sample_y, test_size=0.2, shuffle=False)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
display(X_train , y_train)

,t1_diff_64,t2_diff_64,t3_diff_64,t4_diff_64,t1_diff_63,t2_diff_63,t3_diff_63,t4_diff_63,t1_diff_62,t2_diff_62,...,t3_diff_47,t4_diff_47,t1_diff_46,t2_diff_46,t3_diff_46,t4_diff_46,t1_diff_45,t2_diff_45,t3_diff_45,t4_diff_45
0,0.052485,2.383268,0.025100,15.686275,0.052314,1.789103,-0.004120,7.969925,0.029341,1.579670,...,-11.245059,5.500465,0.017543,0.182667,0.351770,21.641791,0.065871,23.480870,0.364649,5.160116
1,0.052314,1.789103,-0.004120,7.969925,0.029341,1.579670,0.031707,10.361135,0.014799,1.912489,...,0.351770,21.641791,0.065871,23.480870,0.364649,5.160116,0.240064,4.511670,0.105098,15.607581
2,0.029341,1.579670,0.031707,10.361135,0.014799,1.912489,0.083324,18.477594,0.082203,0.363687,...,0.364649,5.160116,0.240064,4.511670,0.105098,15.607581,0.040129,1.849183,0.199680,16.593295
3,0.014799,1.912489,0.083324,18.477594,0.082203,0.363687,-0.035263,6.451613,0.020096,1.664499,...,0.105098,15.607581,0.040129,1.849183,0.199680,16.593295,0.089194,4.564846,0.096678,24.626209
4,0.082203,0.363687,-0.035263,6.451613,0.020096,1.664499,0.031718,11.976048,0.014554,3.314213,...,0.199680,16.593295,0.089194,4.564846,0.096678,24.626209,0.739719,0.948058,2.531423,6.534176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374266,-0.213468,1.186673,1.000105,-0.177141,-0.002389,2.189266,0.550148,-0.351083,-0.002370,4.621514,...,-0.013488,-1.818933,0.018026,1.532148,-0.006387,0.255337,0.009967,5.889145,0.007833,-0.063465
2374267,-0.002389,2.189266,0.550148,-0.351083,-0.002370,4.621514,0.178820,1.381861,-0.089255,-2.664129,...,-0.006387,0.255337,0.009967,5.889145,0.007833,-0.063465,-0.017989,13.891420,0.001057,-8.102564
2374268,-0.002370,4.621514,0.178820,1.381861,-0.089255,-2.664129,-0.067067,-7.507127,0.000000,3.394372,...,0.007833,-0.063465,-0.017989,13.891420,0.001057,-8.102564,0.011015,7.336226,0.008502,2.375201
2374269,-0.089255,-2.664129,-0.067067,-7.507127,0.000000,3.394372,0.064103,1.355276,-0.095353,0.406688,...,0.001057,-8.102564,0.011015,7.336226,0.008502,2.375201,-0.000551,3.338363,-0.096178,0.883392


Index(['t1_diff_64', 't2_diff_64', 't3_diff_64', 't4_diff_64', 't1_diff_63',
       't2_diff_63', 't3_diff_63', 't4_diff_63', 't1_diff_62', 't2_diff_62',
       't3_diff_62', 't4_diff_62', 't1_diff_61', 't2_diff_61', 't3_diff_61',
       't4_diff_61', 't1_diff_60', 't2_diff_60', 't3_diff_60', 't4_diff_60',
       't1_diff_59', 't2_diff_59', 't3_diff_59', 't4_diff_59', 't1_diff_58',
       't2_diff_58', 't3_diff_58', 't4_diff_58', 't1_diff_57', 't2_diff_57',
       't3_diff_57', 't4_diff_57', 't1_diff_56', 't2_diff_56', 't3_diff_56',
       't4_diff_56', 't1_diff_55', 't2_diff_55', 't3_diff_55', 't4_diff_55',
       't1_diff_54', 't2_diff_54', 't3_diff_54', 't4_diff_54', 't1_diff_53',
       't2_diff_53', 't3_diff_53', 't4_diff_53', 't1_diff_52', 't2_diff_52',
       't3_diff_52', 't4_diff_52', 't1_diff_51', 't2_diff_51', 't3_diff_51',
       't4_diff_51', 't1_diff_50', 't2_diff_50', 't3_diff_50', 't4_diff_50',
       't1_diff_49', 't2_diff_49', 't3_diff_49', 't4_diff_49', 't1_diff_48',

,t1_diff,t2_diff,t3_diff,t4_diff
0,0.181572,3.230112,2.159331,2.282504
1,0.040965,4.582807,3.176145,2.226189
2,0.063152,4.376292,3.438099,6.557013
3,0.245613,11.253406,1.254289,9.588672
4,1.282909,54.769294,0.673034,10.498848
...,...,...,...,...
2374266,0.018820,-0.018463,-0.052953,-0.120676
2374267,0.017971,-0.950709,0.011612,-0.502919
2374268,-0.273695,0.741017,-0.008392,-0.144509
2374269,0.011956,31.854310,0.273635,-0.156270


,t1_diff_64,t2_diff_64,t3_diff_64,t4_diff_64,t1_diff_63,t2_diff_63,t3_diff_63,t4_diff_63,t1_diff_62,t2_diff_62,...,t3_diff_47,t4_diff_47,t1_diff_46,t2_diff_46,t3_diff_46,t4_diff_46,t1_diff_45,t2_diff_45,t3_diff_45,t4_diff_45
0,0.052485,2.383268,0.025100,15.686275,0.052314,1.789103,-0.004120,7.969925,0.029341,1.579670,...,-11.245059,5.500465,0.017543,0.182667,0.351770,21.641791,0.065871,23.480870,0.364649,5.160116
1,0.052314,1.789103,-0.004120,7.969925,0.029341,1.579670,0.031707,10.361135,0.014799,1.912489,...,0.351770,21.641791,0.065871,23.480870,0.364649,5.160116,0.240064,4.511670,0.105098,15.607581
2,0.029341,1.579670,0.031707,10.361135,0.014799,1.912489,0.083324,18.477594,0.082203,0.363687,...,0.364649,5.160116,0.240064,4.511670,0.105098,15.607581,0.040129,1.849183,0.199680,16.593295
3,0.014799,1.912489,0.083324,18.477594,0.082203,0.363687,-0.035263,6.451613,0.020096,1.664499,...,0.105098,15.607581,0.040129,1.849183,0.199680,16.593295,0.089194,4.564846,0.096678,24.626209
4,0.082203,0.363687,-0.035263,6.451613,0.020096,1.664499,0.031718,11.976048,0.014554,3.314213,...,0.199680,16.593295,0.089194,4.564846,0.096678,24.626209,0.739719,0.948058,2.531423,6.534176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899411,-0.219392,-4.060914,0.000000,-0.072133,-0.016953,-8.403361,-0.348878,-0.112962,-0.003389,-3.918495,...,0.002394,-0.012935,-0.004393,3.275972,-0.096027,-1.378917,-0.015740,-1.090016,2.707426,-3.701528
1899412,-0.016953,-8.403361,-0.348878,-0.112962,-0.003389,-3.918495,0.000000,-0.169437,0.002002,2.253033,...,-0.096027,-1.378917,-0.015740,-1.090016,2.707426,-3.701528,-2.945555,-17.031259,0.621022,-24.504055
1899413,-0.003389,-3.918495,0.000000,-0.169437,0.002002,2.253033,0.000000,-0.012168,0.000000,2.463768,...,2.707426,-3.701528,-2.945555,-17.031259,0.621022,-24.504055,-0.005463,0.290779,0.064271,-0.014548
1899414,0.002002,2.253033,0.000000,-0.012168,0.000000,2.463768,0.000000,0.000000,0.000000,1.656315,...,0.621022,-24.504055,-0.005463,0.290779,0.064271,-0.014548,-0.003305,-1.656217,-0.054761,-0.361353


,t1_diff,t2_diff,t3_diff,t4_diff
0,0.181572,3.230112,2.159331,2.282504
1,0.040965,4.582807,3.176145,2.226189
2,0.063152,4.376292,3.438099,6.557013
3,0.245613,11.253406,1.254289,9.588672
4,1.282909,54.769294,0.673034,10.498848
...,...,...,...,...
1899411,0.000169,-0.508210,-0.039463,-1.092545
1899412,-4.638753,-7.599550,0.000000,-1.672031
1899413,0.003339,2.830294,-0.002424,-1.242236
1899414,0.083013,0.489711,1.115436,-0.834431


In [62]:
display(X_train.shape, y_train.shape)

(1899416, 80)

(1899416, 4)

In [63]:

# train nueral network 

# if hyper tuning was run , use the optimised paramters found 
acc_per_fold, loss_per_fold , test_results = run_ARNN(X_train, X_test, y_train, y_test, 50, 0.2, 1e-2)


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15
35/35 [==============================] - 1s 19ms/step - loss: 3.0803 - mae: 3.0803
Epoch 2/15
35/35 [==============================] - 0s 14ms/step - loss: 2.9552 - mae: 2.9552
Epoch 3/15
35/35 [==============================] - 0s 13ms/step - loss: 2.9471 - mae: 2.9471
Epoch 4/15
35/35 [==============================] - 0s 14ms/step - loss: 2.9451 - mae: 2.9451
Epoch 5/15
35/35 [==============================] - 0s 14ms/step - loss: 2.9427 - mae: 2.9427
Epoch 6/15
35/35 [==============================] - 1s 15ms/step - loss: 2.9397 - mae: 2.9397
Epoch 7/15
35/35 [==============================] - 0s 14ms/step - loss: 2.9379 - mae: 2.9379
Epoch 8/15
35/35 [==============================] - 0s 14ms/step - loss: 2.9374 - mae: 2.9374
Epoch 9/15
35/35 [==============================] - 0s 14ms/step - loss: 2.9353 - mae: 2.9353
Epoch 10/15
35/35 [==============================] - 1s 1

In [64]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 2.1476101875305176 - Accuracy: 214.76101875305176%
------------------------------------------------------------------------
> Fold 2 - Loss: 2.3405206203460693 - Accuracy: 234.05206203460693%
------------------------------------------------------------------------
> Fold 3 - Loss: 2.1933207511901855 - Accuracy: 219.33207511901855%
------------------------------------------------------------------------
> Fold 4 - Loss: 1.832963228225708 - Accuracy: 183.2963228225708%
------------------------------------------------------------------------
> Fold 5 - Loss: 1.8727171421051025 - Accuracy: 187.27171421051025%
------------------------------------------------------------------------
> Fold 6 - Loss: 1.6385247707366943 - Accuracy: 163.85247707366943%
--------------------------------------------------------------------

In [70]:
#predictions = test_results.predict(X_test)
#display(predictions)
scores = test_results.evaluate(X_test, y_test, verbose=0)
print(f'Score for test {test_results.metrics_names[0]} of {scores[0]}')

Score for test loss of 1.049694538116455


### NN with  5 lags and player box scores merged 

In [ ]:
#run each cell in order 

In [76]:
#building the data again but with 20 lags 
targets = reset()
# collecting what we need 
trgt_lags = targets
day_back(trgt_lags)

In [79]:
#trgt_lags.reset_index()
trgt_lags 

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,...,t3_diff_3,t4_diff_3,t1_diff_4,t2_diff_4,t3_diff_4,t4_diff_4,t1_diff_5,t2_diff_5,t3_diff_5,t4_diff_5
11143,2018-01-06,112526,0.020096,2.054616,0.035855,12.147134,5,20180106,0.020096,1.664499,...,0.031707,10.361135,0.052314,1.789103,-0.004120,7.969925,0.052485,2.383268,0.025100,15.686275
13182,2018-01-07,112526,0.014554,4.716380,0.026337,16.176471,6,20180107,0.014554,3.314213,...,0.083324,18.477594,0.029341,1.579670,0.031707,10.361135,0.052314,1.789103,-0.004120,7.969925
15406,2018-01-08,112526,0.032364,4.396423,0.039198,17.619048,7,20180108,0.032364,1.788376,...,-0.035263,6.451613,0.014799,1.912489,0.083324,18.477594,0.029341,1.579670,0.031707,10.361135
18008,2018-01-09,112526,0.016104,1.791045,0.055419,4.627697,8,20180109,0.016104,-1.028192,...,0.031718,11.976048,0.082203,0.363687,-0.035263,6.451613,0.014799,1.912489,0.083324,18.477594
18617,2018-01-10,112526,0.075636,1.866901,0.023244,8.506344,9,20180110,0.019394,0.549089,...,0.006584,15.866873,0.020096,1.664499,0.031718,11.976048,0.082203,0.363687,-0.035263,6.451613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495980,2021-04-26,685503,0.044617,1.224728,0.009437,0.737463,1211,20210426,0.018820,-0.018463,...,-0.051184,-2.655033,0.099302,-0.373574,0.007259,-1.477378,0.121678,1.688486,0.157717,-0.092081
2497962,2021-04-27,685503,0.019123,1.178880,0.013161,0.790301,1212,20210427,0.017971,-0.950709,...,-1.471642,-0.055074,-0.062143,-3.985773,-0.051184,-2.655033,0.099302,-0.373574,0.007259,-1.477378
2500311,2021-04-28,685503,0.015799,4.323489,0.002350,0.970273,1213,20210428,-0.273695,0.741017,...,0.014520,-0.006554,-0.063314,14.899376,-1.471642,-0.055074,-0.062143,-3.985773,-0.051184,-2.655033
2502400,2021-04-29,685503,0.018770,31.946021,0.305491,5.938273,1214,20210429,0.011956,31.854310,...,-0.052953,-0.120676,0.075090,1.644315,0.014520,-0.006554,-0.063314,14.899376,-1.471642,-0.055074


In [126]:
%%time
#building our feature set ...
lags = 5 
offset = 1 

trgt_lags, FECOLS = create_lags_dataset(trgt_lags , lags, offset)
trgt_lags = trgt_lags.sort_values(by=["playerId", "engagementMetricsDate"])
print('train sorted---')
display(trgt_lags)
trgt_lags = trgt_lags.dropna()
print(trgt_lags.shape)
gc.collect()

train sorted---


,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,...,t3_diff_3,t4_diff_3,t1_diff_4,t2_diff_4,t3_diff_4,t4_diff_4,t1_diff_5,t2_diff_5,t3_diff_5,t4_diff_5
0,2018-03-06,112526,0.189106,6.780566,2.209322,2.322992,64,20180306,0.181572,3.230112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-03-07,112526,0.053513,4.867257,3.300729,2.300283,65,20180307,0.040965,4.582807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-03-08,112526,0.066996,11.750517,3.492547,7.134919,66,20180308,0.063152,4.376292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-03-09,112526,0.249823,18.964578,1.259076,9.912340,67,20180309,0.245613,11.253406,...,2.159331,2.282504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-03-10,112526,1.302126,56.259052,1.190449,10.539515,68,20180310,1.282909,54.769294,...,3.176145,2.226189,0.181572,3.230112,2.159331,2.282504,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374266,2021-04-26,685503,0.044617,1.224728,0.009437,0.737463,1211,20210426,0.018820,-0.018463,...,-0.051184,-2.655033,0.099302,-0.373574,0.007259,-1.477378,0.121678,1.688486,0.157717,-0.092081
2374267,2021-04-27,685503,0.019123,1.178880,0.013161,0.790301,1212,20210427,0.017971,-0.950709,...,-1.471642,-0.055074,-0.062143,-3.985773,-0.051184,-2.655033,0.099302,-0.373574,0.007259,-1.477378
2374268,2021-04-28,685503,0.015799,4.323489,0.002350,0.970273,1213,20210428,-0.273695,0.741017,...,0.014520,-0.006554,-0.063314,14.899376,-1.471642,-0.055074,-0.062143,-3.985773,-0.051184,-2.655033
2374269,2021-04-29,685503,0.018770,31.946021,0.305491,5.938273,1214,20210429,0.011956,31.854310,...,-0.052953,-0.120676,0.075090,1.644315,0.014520,-0.006554,-0.063314,14.899376,-1.471642,-0.055074


(2363966, 112)
train sorted---


,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,t1_diff,t2_diff,...,t3_diff_3,t4_diff_3,t1_diff_4,t2_diff_4,t3_diff_4,t4_diff_4,t1_diff_5,t2_diff_5,t3_diff_5,t4_diff_5
5,2018-03-11,112526,2.914909,5.981786,2.235915,8.608041,69,20180311,2.587557,4.322605,...,3.438099,6.557013,0.040965,4.582807,3.176145,2.226189,0.181572,3.230112,2.159331,2.282504
6,2018-03-12,112526,0.241417,7.036852,0.446483,11.322689,70,20180312,0.240733,3.835251,...,1.254289,9.588672,0.063152,4.376292,3.438099,6.557013,0.040965,4.582807,3.176145,2.226189
7,2018-03-13,112526,0.187200,5.783341,0.485713,18.297456,71,20180313,0.158605,1.359517,...,0.673034,10.498848,0.245613,11.253406,1.254289,9.588672,0.063152,4.376292,3.438099,6.557013
8,2018-03-14,112526,1.875354,20.172911,61.578659,35.825840,72,20180314,1.872120,18.395773,...,2.009916,8.422185,1.282909,54.769294,0.673034,10.498848,0.245613,11.253406,1.254289,9.588672
9,2018-03-15,112526,0.257301,7.236447,22.043605,11.399549,73,20180315,-34.024533,0.456786,...,0.443780,11.205508,2.587557,4.322605,2.009916,8.422185,1.282909,54.769294,0.673034,10.498848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374266,2021-04-26,685503,0.044617,1.224728,0.009437,0.737463,1211,20210426,0.018820,-0.018463,...,-0.051184,-2.655033,0.099302,-0.373574,0.007259,-1.477378,0.121678,1.688486,0.157717,-0.092081
2374267,2021-04-27,685503,0.019123,1.178880,0.013161,0.790301,1212,20210427,0.017971,-0.950709,...,-1.471642,-0.055074,-0.062143,-3.985773,-0.051184,-2.655033,0.099302,-0.373574,0.007259,-1.477378
2374268,2021-04-28,685503,0.015799,4.323489,0.002350,0.970273,1213,20210428,-0.273695,0.741017,...,0.014520,-0.006554,-0.063314,14.899376,-1.471642,-0.055074,-0.062143,-3.985773,-0.051184,-2.655033
2374269,2021-04-29,685503,0.018770,31.946021,0.305491,5.938273,1214,20210429,0.011956,31.854310,...,-0.052953,-0.120676,0.075090,1.644315,0.014520,-0.006554,-0.063314,14.899376,-1.471642,-0.055074


(2363966, 112)
CPU times: user 32.7 s, sys: 1min 15s, total: 1min 48s
Wall time: 3min 17s


0

In [80]:
pl_eng_w_scores.fillna(-1,inplace = True)
display(pl_eng_w_scores)

,engagementMetricsDate,playerId,target1,target2,target3,target4,index_x,date,home,gamePk,...,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,index_y
0,2018-03-30,608365,2.040838,7.893376,0.052359,2.599964,87,20180329,0,529407,...,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,3.0,1.0,9.0,87
1,2018-03-30,502624,1.068945,5.105543,2.722191,0.918550,87,20180329,0,529413,...,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,87
2,2018-03-30,643338,0.385945,5.368812,0.074404,0.565661,87,20180329,0,529415,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,87
3,2018-03-30,570560,0.131860,6.200931,0.002296,0.181634,87,20180329,0,529408,...,-1.0,-1.0,-1.0,-1.0,-1.0,4.0,1.0,0.0,5.0,87
4,2018-03-30,523260,0.011800,9.933713,6.076820,0.978230,87,20180329,0,529406,...,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184754,2021-05-01,664057,0.002498,1.963852,0.000000,1.285991,1215,20210430,1,634274,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,1215
184755,2021-05-01,664854,0.446403,0.625770,0.000000,0.273131,1215,20210430,0,634325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1215
184756,2021-05-01,605612,0.016699,1.198282,0.000000,0.739729,1215,20210430,0,634327,...,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,2.0,0.0,2.0,1215
184757,2021-05-01,643778,0.000000,0.595813,0.000000,0.273131,1215,20210430,0,634325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1215


In [81]:
lag_with_scores = pd.merge(trgt_lags,pl_eng_w_scores,on=['playerId','date','target1', 'target2', 'target3', 'target4' ],how = 'left').reset_index()

In [82]:
display(lag_with_scores)
display(lag_with_scores.columns.tolist())

,level_0,engagementMetricsDate_x,playerId,target1,target2,target3,target4,index,date,t1_diff,...,sacBuntsPitching,sacFliesPitching,saves,holds,blownSaves,assists,putOuts,errors,chances,index_y
0,0,2018-01-06,112526,0.020096,2.054616,0.035855,12.147134,5,20180106,0.020096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-01-07,112526,0.014554,4.716380,0.026337,16.176471,6,20180107,0.014554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018-01-08,112526,0.032364,4.396423,0.039198,17.619048,7,20180108,0.032364,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2018-01-09,112526,0.016104,1.791045,0.055419,4.627697,8,20180109,0.016104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2018-01-10,112526,0.075636,1.866901,0.023244,8.506344,9,20180110,0.019394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498048,2498048,2021-04-26,685503,0.044617,1.224728,0.009437,0.737463,1211,20210426,0.018820,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2498049,2498049,2021-04-27,685503,0.019123,1.178880,0.013161,0.790301,1212,20210427,0.017971,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2498050,2498050,2021-04-28,685503,0.015799,4.323489,0.002350,0.970273,1213,20210428,-0.273695,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2498051,2498051,2021-04-29,685503,0.018770,31.946021,0.305491,5.938273,1214,20210429,0.011956,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['level_0',
 'engagementMetricsDate_x',
 'playerId',
 'target1',
 'target2',
 'target3',
 'target4',
 'index',
 'date',
 't1_diff',
 't2_diff',
 't3_diff',
 't4_diff',
 't1_diff_1',
 't2_diff_1',
 't3_diff_1',
 't4_diff_1',
 't1_diff_2',
 't2_diff_2',
 't3_diff_2',
 't4_diff_2',
 't1_diff_3',
 't2_diff_3',
 't3_diff_3',
 't4_diff_3',
 't1_diff_4',
 't2_diff_4',
 't3_diff_4',
 't4_diff_4',
 't1_diff_5',
 't2_diff_5',
 't3_diff_5',
 't4_diff_5',
 'engagementMetricsDate_y',
 'index_x',
 'home',
 'gamePk',
 'gameDate',
 'gameTimeUTC',
 'teamId',
 'teamName',
 'playerName',
 'jerseyNum',
 'positionCode',
 'positionName',
 'positionType',
 'battingOrder',
 'gamesPlayedBatting',
 'flyOuts',
 'groundOuts',
 'runsScored',
 'doubles',
 'triples',
 'homeRuns',
 'strikeOuts',
 'baseOnBalls',
 'intentionalWalks',
 'hits',
 'hitByPitch',
 'atBats',
 'caughtStealing',
 'stolenBases',
 'groundIntoDoublePlay',
 'groundIntoTriplePlay',
 'plateAppearances',
 'totalBases',
 'rbi',
 'leftOnBase',
 'sacBunt

In [92]:
##### notice how the number of rows is now 184759 from 2.5mill+ because were only taking those rows are that in 
##### playerbox info

In [83]:
lag_with_scores = lag_with_scores.dropna()
print(lag_with_scores.shape)

(184759, 120)


In [84]:
lag_with_scores.fillna(-1,inplace = True)

sample_y = lag_with_scores[['t1_diff','t2_diff', 't3_diff', 't4_diff']]
sample_y = sample_y.reset_index(drop=True)
extra =["hits",'runsScored', 'homeRuns','gamesStartedPitching', 'strikeOuts','stolenBases','homeRunsPitching',"strikeOutsPitching"]

sample_X = lag_with_scores[FECOLS + extra]
sample_X = sample_X.reset_index(drop=True)

#display(sample_X , sample_X.columns)
#display(sample_y)


from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(sample_X, sample_y, test_size=0.2, shuffle=False)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
display(X_train , y_train)

,t1_diff_5,t2_diff_5,t3_diff_5,t4_diff_5,t1_diff_4,t2_diff_4,t3_diff_4,t4_diff_4,t1_diff_3,t2_diff_3,...,t3_diff_1,t4_diff_1,hits,runsScored,homeRuns,gamesStartedPitching,strikeOuts,stolenBases,homeRunsPitching,strikeOutsPitching
0,0.033605,2.717271,0.556309,6.320883,0.031466,2.040337,0.378452,6.305301,0.023954,4.454647,...,7.564146,99.971645,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,4.0
1,0.246430,0.869100,2.557019,6.955363,0.741592,3.517105,5.851915,10.161356,0.198644,3.406773,...,0.345038,15.278360,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0,4.0
2,0.198644,3.406773,6.375189,12.164923,0.076686,1.448902,2.733334,7.015608,0.231458,4.016754,...,15.382995,25.320300,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0,2.0
3,24.790357,15.720602,37.622677,47.129348,0.625645,2.863581,0.918781,11.003200,0.205808,3.192877,...,31.950883,99.739137,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,0.0,7.0
4,1.541581,7.152441,42.738173,42.638558,0.538553,6.309306,14.718143,10.439724,0.313252,5.641120,...,1.088716,22.257667,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147802,2.203188,-14.327171,-2.388824,-1.655039,-4.072807,-2.353946,4.304377,-0.037517,0.185482,-17.255034,...,11.136512,1.151601,1.0,1.0,1.0,-1.0,0.0,0.0,-1.0,-1.0
147803,-4.072807,-2.353946,4.304377,-0.037517,0.185482,-17.255034,0.650402,-2.249226,0.943827,6.547201,...,14.710099,-11.965903,0.0,0.0,0.0,-1.0,2.0,0.0,-1.0,-1.0
147804,0.185482,-17.255034,0.650402,-2.249226,0.943827,6.547201,11.193822,2.942031,5.195106,9.092943,...,0.531332,-0.459594,0.0,1.0,0.0,-1.0,1.0,0.0,-1.0,-1.0
147805,0.943827,6.547201,11.193822,2.942031,5.195106,9.092943,11.136512,1.151601,6.564900,-22.382527,...,0.140040,1.245043,1.0,1.0,1.0,-1.0,2.0,0.0,-1.0,-1.0


,t1_diff,t2_diff,t3_diff,t4_diff
0,5.957764,16.498360,33.762162,35.874223
1,1.115075,3.659805,8.403585,5.896739
2,24.790357,15.720602,37.622677,47.129348
3,61.485526,69.890278,99.999656,99.861874
4,7.161259,16.604872,70.042112,22.693430
...,...,...,...,...
147802,6.564900,-22.382527,14.710099,-11.965903
147803,0.463917,2.628091,0.531332,-0.459594
147804,0.420817,7.855684,0.140040,1.245043
147805,1.797024,2.738476,0.181142,-0.194076


In [85]:
# train nueral network 
acc_per_fold, loss_per_fold , test_results = run_ARNN(X_train, X_test, y_train, y_test, 50, 0.2, 1e-2)

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15
3/3 [==============================] - 1s 16ms/step - loss: 5.9361 - mae: 5.9361
Epoch 2/15
3/3 [==============================] - 0s 20ms/step - loss: 5.6745 - mae: 5.6745
Epoch 3/15
3/3 [==============================] - 0s 18ms/step - loss: 5.5450 - mae: 5.5450
Epoch 4/15
3/3 [==============================] - 0s 17ms/step - loss: 5.4213 - mae: 5.4213
Epoch 5/15
3/3 [==============================] - 0s 16ms/step - loss: 5.3216 - mae: 5.3216
Epoch 6/15
3/3 [==============================] - 0s 15ms/step - loss: 5.2162 - mae: 5.2162
Epoch 7/15
3/3 [==============================] - 0s 16ms/step - loss: 5.1284 - mae: 5.1284
Epoch 8/15
3/3 [==============================] - 0s 16ms/step - loss: 5.0587 - mae: 5.0587
Epoch 9/15
3/3 [==============================] - 0s 17ms/step - loss: 5.0304 - mae: 5.0304
Epoch 10/15
3/3 [==============================] - 0s 18ms/step - loss: 5.0

In [86]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 5.736974716186523 - Accuracy: 573.6974716186523%
------------------------------------------------------------------------
> Fold 2 - Loss: 4.170536518096924 - Accuracy: 417.0536518096924%
------------------------------------------------------------------------
> Fold 3 - Loss: 4.574608325958252 - Accuracy: 457.4608325958252%
------------------------------------------------------------------------
> Fold 4 - Loss: 3.945234775543213 - Accuracy: 394.5234775543213%
------------------------------------------------------------------------
> Fold 5 - Loss: 4.441318988800049 - Accuracy: 444.1318988800049%
------------------------------------------------------------------------
> Fold 6 - Loss: 4.185070991516113 - Accuracy: 418.5070991516113%
------------------------------------------------------------------------
> Fol

In [87]:
#predictions = test_results.predict(X_test)
#display(predictions)
scores = test_results.evaluate(X_test, y_test, verbose=0)
print(f'Score for test {test_results.metrics_names[0]} of {scores[0]}')

Score for test loss of 3.8613407611846924


### Plots for data Exploration 

#### Search for optimal time lag 

In [3]:
#add mean useful for analysis later 
targets = pd.read_pickle("/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/nextDayPlayerEngagement_train.pkl")

targets['targetAvg'] = np.mean(
targets[['target1', 'target2', 'target3', 'target4']],
axis = 1)
targets["engagementMetricsDate"] = pd.to_datetime(targets["engagementMetricsDate"])

In [65]:
# cell 58 & 59 are courtsey of https://www.kaggle.com/arpitsolanki14/mlb-digital-engagement-data-deep-dive/notebook

autocorrel_list=list()
for i in range(25):
    ser=targets.groupby('playerId')['targetAvg'].apply(lambda x: x.autocorr(lag=i))
    df=ser.to_frame().reset_index()
    df['lag']=i
    autocorrel_list.append(df)
auto_frame=pd.concat(autocorrel_list).reset_index(drop=True)



In [66]:
lag_gp=auto_frame.groupby('lag').agg({'targetAvg':['mean','median']}).reset_index()
lag_gp.columns=['lag','mean','median']
fig=go.Figure()
fig.add_trace(go.Scatter(x=lag_gp['lag'],y=lag_gp['mean'],mode='lines',name='mean'))
fig.add_trace(go.Scatter(x=lag_gp['lag'],y=lag_gp['median'],mode='lines',name='median'))
fig.update_layout(title='Mean & Median of player Autocorrelation distributions across various lag periods', title_x=0.5 ,xaxis_title='lag')

#### Exploring player position and target average relationship 

In [2]:
players = pd.read_csv('/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/players.csv')

In [3]:
display(players)

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,False
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,True
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2056,672695,Geraldo Perdomo,1999-10-22,NaN,Santo Domingo,NaN,Dominican Republic,74,203,6,Shortstop,True
2057,672911,Jesus Cruz,1995-04-15,NaN,Salinas de Hidalgo,San Luis Potosi,Mexico,73,230,1,Pitcher,False
2058,676103,Damon Jones,1994-09-30,NaN,Twin Falls,ID,USA,77,233,1,Pitcher,True
2059,676755,Isaac Mattson,1995-07-14,NaN,Erie,PA,USA,74,205,1,Pitcher,True


In [18]:
players_info=players[['playerId','playerName','primaryPositionName']]

In [4]:
targets = pd.read_pickle("/Users/daksh/Desktop/sem2-2021/COMP9417-2021/project/mlb-player-digital-engagement-forecasting/nextDayPlayerEngagement_train.pkl")
targets['targetAvg'] = np.mean(
targets[['target1', 'target2', 'target3', 'target4']],
axis = 1)
targets["engagementMetricsDate"] = pd.to_datetime(targets["engagementMetricsDate"])

In [138]:
plys_trtg = pd.merge(targets,players,on=['playerId'],how = 'left')

In [26]:
display(plys_trtg)

,engagementMetricsDate,playerId,target1,target2,target3,target4,index,date,targetAvg,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294,0,20180101,2.556584,Kenta Maeda,1988-04-11,2016-04-06,Osaka,NaN,Japan,73,185,1,Pitcher,True
1,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118,0,20180101,2.118882,Jose Abreu,1987-01-29,2014-03-31,Cienfuegos,NaN,Cuba,75,235,3,First Base,True
2,2018-01-02,519317,0.974327,56.177043,13.693746,64.166667,0,20180101,33.752945,Giancarlo Stanton,1989-11-08,2010-06-08,Panorama,CA,USA,78,245,10,Designated Hitter,True
3,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745,0,20180101,1.137428,Seth Lugo,1989-11-17,2016-07-01,Shreveport,LA,USA,76,225,1,Pitcher,True
4,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373,0,20180101,0.391934,Deck McGuire,1989-06-23,2017-09-12,Richmond,VA,USA,78,220,1,Pitcher,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2506171,2021-05-01,451661,0.000000,0.013314,0.000000,0.625925,1215,20210430,0.159810,Josh Fields,1985-08-19,2013-04-02,Athens,GA,USA,72,191,1,Pitcher,False
2506172,2021-05-01,519301,0.000131,0.003329,0.000000,0.216229,1215,20210430,0.054922,Sammy Solis,1988-08-10,2015-04-30,Minneapolis,MN,USA,77,251,1,Pitcher,False
2506173,2021-05-01,527055,0.000000,0.019971,0.000000,0.273131,1215,20210430,0.073276,Arodys Vizcaino,1990-11-13,2011-08-10,Yaguate,NaN,Dominican Republic,72,245,1,Pitcher,False
2506174,2021-05-01,543484,0.000131,0.056586,0.000000,1.024240,1215,20210430,0.270239,Mikie Mahtook,1989-11-30,2015-04-10,Lafayette,LA,USA,73,216,8,Outfielder,False


In [12]:
player_high=plys_trtg.groupby('playerId').agg({'targetAvg':'mean'}).reset_index()
player_high.sort_values(by='targetAvg',inplace=True,ascending=False)

In [14]:
player_high3 = player_high.head(3)

In [15]:
top3_players=plys_trtg.loc[plys_trtg['playerId'].isin(player_high3.playerId)]

In [25]:
player_high100=player_high.head(100)

top100_players=pd.merge(left=player_high100,right=players_info,left_on='playerId',right_on='playerId',how='inner')
top100_players

fig=px.scatter(top100_players,y='targetAvg',color='primaryPositionName',title='TargetAvg scores for top 100 players')
fig.update_layout(title_x=0.5)
fig.show()

fig1 = px.pie(top100_players, values='targetAvg', names='primaryPositionName', title='Breakdown of top 100 players by position')
fig1.update_layout(title_x=0.5)
fig1.show()

In [28]:
position_avg=plys_trtg.groupby(['primaryPositionName']).agg({'targetAvg':'mean','playerId':'nunique'}).reset_index()
position_avg.sort_values(by='targetAvg',inplace=True,ascending=False)

fig=px.bar(position_avg,x='primaryPositionName',y='targetAvg',title='Target Avg values by Position')
fig.update_layout(title_x=0.5)
fig.show()
position_avg.sort_values(by='playerId',inplace=True,ascending=False)

fig1=px.bar(position_avg,x='primaryPositionName',y='playerId',title='Count of Players by Position')
fig1.update_layout(title_x=0.5)
fig1.show()


player_high_join=pd.merge(left=player_high,right=players,how='inner')
#player_high_join=player_high_join.loc[player_high_join['primaryPositionName']=='Pitcher']
player_high_join

fig2 = px.violin(player_high_join,y='targetAvg',box=True,x='primaryPositionName',title='Distribution of targetAvg by primaryPosition')
fig2.update_layout(title_x=0.5)
fig2.show()

In [12]:
# merge target vaerage on to player box scores group by player 
extra =["playerId","targetAvg", "hits",'runsScored', 'homeRuns','gamesStartedPitching', 'strikeOuts','stolenBases','homeRunsPitching',"strikeOutsPitching"]
Box_features=pl_eng_w_scores[extra]
Box_features = Box_features.dropna()

Box_features=Box_features.groupby('playerId', '').agg({'targetAvg':'mean'}).reset_index()
player_high.sort_values(by='targetAvg',inplace=True,ascending=False)

# box_scores_corr = box_scores.corr()
# fig = px.imshow(box_scores_corr)
# fig.show()

,playerId,targetAvg,hits,runsScored,homeRuns,gamesStartedPitching,strikeOuts,stolenBases,homeRunsPitching,strikeOutsPitching
1,502624,2.453807,1.0,1.0,0.0,1.0,1.0,0.0,0.0,6.0
21,518715,1.022576,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
32,458677,3.073386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
33,547749,0.658068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
41,622217,1.036933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
184722,668941,0.357930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184741,641755,0.202412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184743,650893,1.929128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
184755,664854,0.336326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
pl_eng_w_scores.columns

Index(['engagementMetricsDate', 'playerId', 'target1', 'target2', 'target3',
       'target4', 'index_x', 'date', 'targetAvg', 'home', 'gamePk', 'gameDate',
       'gameTimeUTC', 'teamId', 'teamName', 'playerName', 'jerseyNum',
       'positionCode', 'positionName', 'positionType', 'battingOrder',
       'gamesPlayedBatting', 'flyOuts', 'groundOuts', 'runsScored', 'doubles',
       'triples', 'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks',
       'hits', 'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases',
       'groundIntoDoublePlay', 'groundIntoTriplePlay', 'plateAppearances',
       'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies',
       'catchersInterference', 'pickoffs', 'gamesPlayedPitching',
       'gamesStartedPitching', 'completeGamesPitching', 'shutoutsPitching',
       'winsPitching', 'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching',

In [17]:
run_stats=pl_eng_w_scores.groupby(['runsScored','homeRuns']).agg({'targetAvg':'mean','target1':'count'}).reset_index()
run_stats['homeRuns']=run_stats['homeRuns'].apply(str)
run_stats.columns=['runsScored','homeRuns','targetAvg','Innings']


#team_run_stats

In [20]:
fig = px.bar(run_stats, x="runsScored", y="targetAvg",
             color='homeRuns', barmode='group',
             height=400)
fig.update_layout(title='Relationship between targetAvg with Player runsScored & homeRuns scored in Match')
fig.update_layout(title_x=0.5)
fig.show()
